In [ ]:
NAME = "Alasdair Breasley"
CIS_USERNAME = "fmcv76"

---

# COMP42415 Text Mining and Language Analytics
## Coursework 2023-24

### Instructions:

- <ins>**DO NOT RENAME THIS JUPYTER NOTEBOOK !!!**</ins>
- Please write the answers for each question in the respective cell. 
- You can add more cells if needed.
- If needed, you can upload additional files, e.g. pre-trained word embeddings, in the coursework's directory
- You can safely remove the `raise NotImplementedError()` line from each code cell.
- You can find information about markdown syntax from [here](https://www.markdownguide.org/basic-syntax/#emphasis).

In [ ]:
# Import required packages 
import numpy as np
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
nltk.download("averaged_perceptron_tagger")
from nltk import pos_tag
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score,precision_score, recall_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
sns.set() # Use seaborn plotting style
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import gensim
from gensim.models import Word2Vec
import ast
import pickle
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Controls 
load_preprocessed_data = True
load_precalculated_w2v_embeddings_300 = True
load_precalculated_w2v_docs_vectorised = True
load_precalculated_w2v_embeddings_50 = True
load_precalculated_w2v_words_vectorised = True

load_NB = True
run_kNN = False
run_kNN_cross_validation = False
load_kNN = True
load_CNN = True
load_LSTM = True


## Implementation - Task 1
Prepare the dataset by applying any pre-processing or cleaning steps that you consider as necessary. Then, split the dataset into a training set containing 70% of the samples and a test set containing 30% of the samples. Follow an appropriate strategy for the split. You must use these training/test sets for all the models in this coursework. (**10%**)

In [ ]:
if (load_preprocessed_data != True):
    
    # Import food reviews data 
    food_reviews_data = pd.read_csv("food_reviews.csv")
    
    # Check dataframe head 
    print(food_reviews_data.head(5), "\n")
    
    # Check dataframe shape 
    print(food_reviews_data.shape, "\n")
    
    # Check data types 
    print(food_reviews_data.dtypes, "\n")
    
    # Check for missing values 
    print(food_reviews_data.isna().sum())
    
    # Check uniqueness of Score column 
    print(food_reviews_data["Score"].unique(), "\n")
    
    # Check for duplicate rows 
    print(food_reviews_data[food_reviews_data.duplicated()].shape[0], "\n")
    
    # Remove duplicate rows 
    food_reviews_data = food_reviews_data.drop_duplicates()
    
    # Recheck dataframe shape 
    print(food_reviews_data.shape, "\n")
    
    # Concatenate Summary column with Text column to capture all available information 
    food_reviews_data["Text"] = food_reviews_data["Summary"].astype(str) + " " + food_reviews_data["Text"].astype(str)
    
    # Check how many rows contain html tags 
    print(food_reviews_data["Text"].str.contains(r"<.+?>").sum(), "\n")
    
    # Replace html tags with " ", " " as opposed to "" to not join words unintentionally 
    food_reviews_data["Text"] = food_reviews_data["Text"].str.replace(r"<.+?>", " ", regex = True)
    
    # Recheck how many rows contain html tags 
    print(food_reviews_data["Text"].str.contains(r"<.+?>").sum(), "\n")
    
    # Transform Text column to all lowercase 
    food_reviews_data["Text"] = food_reviews_data["Text"].str.lower()
    
    # Remove all "-", ".", "'" and replace with "" e.g. lower-case to lowercase, U.K to UK, don't to dont 
    food_reviews_data["Text"] = food_reviews_data["Text"].str.replace(r"[-.']", "", regex = True)
    
    # Check how many rows include happy or sad text emojis 
    print(food_reviews_data["Text"].str.contains(r"[:][)]|[:][(]|[)][:]|[(][:]").sum(), "\n")
    
    # Replace happy emojis with " happy " 
    food_reviews_data["Text"] = food_reviews_data["Text"].str.replace(r"[:][)]|[(][:]", " happy ", regex = True)
    
    # Replace sad emojis with " sad " 
    food_reviews_data["Text"] = food_reviews_data["Text"].str.replace(r"[:][(]|[)][:]", " sad ", regex = True)
    
    # Recheck how many rows include happy or sad text emojis 
    print(food_reviews_data["Text"].str.contains(r"[:][)]|[:][(]|[)][:]|[(][:]").sum(), "\n")
    
    # Check how many rows include punctuation 
    print(food_reviews_data["Text"].str.contains(r"[^\w\s]").sum(), "\n")

    # Remove punctuation 
    food_reviews_data["Text"] = food_reviews_data["Text"].str.replace(r"[^\w\s]", " ", regex = True)
    
    # Recheck how many rows include punctuation 
    print(food_reviews_data["Text"].str.contains(r"[^\w\s]").sum(), "\n")
    
    # Check for class imbalance 
    print(food_reviews_data["Score"].value_counts(), "\n")
    
    # There is class imbalance present 
    
    # Check dataframe head 
    print(food_reviews_data.head(5), "\n")


In [ ]:
def penn_to_wordnet(penn_pos_tag):
    """Function to convert Penn Treeback POS tags to WordNet."""
    
    tag_dictionary = {"NN":"n", "JJ":"a","VB":"v", "RB":"r"}
    
    try:
        
        # If the first two characters of the Penn Treebank POS tag are in the tag_dictionary 
        return tag_dictionary[penn_pos_tag[:2]]
    
    except:
        
        return "n" # Default to Noun if no mapping available 
    

# Get list of English stop words - Define outside of function so doesn't have to process every call 
stopwords_english = stopwords.words("english")


def remove_stopwords_and_lemmatise(text):
    """Function to tokenise a string, remove stop words, lemmatise, and return a list of words."""
    
    # Check text input is a string 
    if (not isinstance(text, str)):
        
        raise Exception("text input is not a string.")
    
    # Tokenise text into words 
    words = word_tokenize(text)
    
    # Remove stop words 
    words_filtered = [word for word in words if word not in stopwords_english]
    
    # TODO: Update to use full conversion table as opposed to simplified version 
    
    # Apply POS tagging 
    words_pos_tagged = pos_tag(words_filtered)
    
    # Create a WordNetLemmatizer object 
    wnl = WordNetLemmatizer()
    
    # Define empty lemmas list 
    lemmas = []
    
    # Loop through words in sentence and lemmatise 
    for word, tag in words_pos_tagged:
        
        lemmas.append(wnl.lemmatize(word, pos = penn_to_wordnet(tag)))
    
    return lemmas
    

In [ ]:
if (load_preprocessed_data != True):
        
    # Split data into training (70%) and test (30%) splits 
    train_data, test_data, train_labels, test_labels = train_test_split(food_reviews_data["Text"], food_reviews_data["Score"], 
                                                                        test_size = 0.3, random_state = 123, 
                                                                        stratify = food_reviews_data["Score"])
    
    train_labels = train_labels.tolist()
    
    test_labels = test_labels.tolist()
    
    # TODO: Decide if it should be stratified random sampling or random sampling 
    
    # TODO: Remove the seed as it should not be used for real evaluation of model performance 
    
    # Remove stop words from train data and lemmatise 
    train_data_tokenised = train_data.apply(remove_stopwords_and_lemmatise)
    
    # Join lists of lemmas to string (need both ways) 
    train_data = [" ".join(lemmas) for lemmas in train_data_tokenised]
    
    # Remove stop words from test data and lemmatise 
    test_data_tokenised = test_data.apply(remove_stopwords_and_lemmatise)
    
    # Join lists of lemmas to string (need both ways) 
    test_data = [" ".join(lemmas) for lemmas in test_data_tokenised]
    
    # Save tokenised train data to csv 
    train_data_tokenised_csv = pd.Series(train_data_tokenised)
    train_data_tokenised_csv.to_csv("train_data_tokenised.csv")
    
    # Save train data to csv 
    train_data_csv = pd.DataFrame({"Text" : train_data, 
                                   "Score" : train_labels})
    train_data_csv.to_csv("train_data.csv")
    
    # Save tokenised test data to csv 
    test_data_tokenised_csv = pd.Series(test_data_tokenised)
    test_data_tokenised_csv.to_csv("test_data_tokenised.csv")
    
    # Save test data to csv 
    test_data_csv = pd.DataFrame({"Text" : test_data, 
                                  "Score" : test_labels})
    test_data_csv.to_csv("test_data.csv")
    
if (load_preprocessed_data == True):
    
    # Load preprocessed tokenised train data 
    train_data_tokenised = pd.read_csv("train_data_tokenised.csv", index_col = 0)["Text"].tolist()
    train_data_tokenised = [ast.literal_eval(text) for text in train_data_tokenised]
    
    # Load preprocessed train data 
    train_data = pd.read_csv("train_data.csv", index_col = 0)["Text"].tolist()
    
    # Load train labels 
    train_labels = pd.read_csv("train_data.csv", index_col = 0)["Score"].tolist()
    
    # Load preprocessed tokenised test data 
    test_data_tokenised = pd.read_csv("test_data_tokenised.csv", index_col = 0)["Text"].tolist()
    test_data_tokenised = [ast.literal_eval(text) for text in test_data_tokenised]
    
    # Load preprocessed test data 
    test_data = pd.read_csv("test_data.csv", index_col = 0)["Text"].tolist()
    
    # Load train labels 
    test_labels = pd.read_csv("test_data.csv", index_col = 0)["Score"].tolist()
    
# TODO: Check if any list items are empty strings 
    
# TODO: Save using pickle instead of csv 


In [ ]:
# Visualise the frequency of Score values for the entire dataset 
all_scores = pd.Series(train_labels + test_labels)
all_frequencies = all_scores.value_counts()
all_frequencies = all_frequencies.reset_index()
all_frequencies.columns = ["Score", "Frequency"]

plt.figure(figsize = (6, 4), dpi = 100)
plt.bar(all_frequencies["Score"], all_frequencies["Frequency"], color = "skyblue", edgecolor = "black")
plt.xlabel("Score")
plt.ylabel("Frequency")
plt.title("Frequency of Food Review Scores - Full Dataset Deduplicated")
ax = plt.gca()
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
plt.savefig("Figure_1.png")
plt.show()


In [ ]:
# Visualise the frequency of Score values for the train dataset 
train_scores = pd.Series(train_labels)
train_frequencies = train_scores.value_counts()
train_frequencies = train_frequencies.reset_index()
train_frequencies.columns = ["Score", "Frequency"]

plt.figure(figsize = (6, 4), dpi = 100)
plt.bar(train_frequencies["Score"], train_frequencies["Frequency"], color = "skyblue", edgecolor = "black")
plt.xlabel("Score")
plt.ylabel("Frequency")
plt.title("Frequency of Food Review Scores - Train Dataset")
ax = plt.gca()
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
plt.show()


In [ ]:
# Visualise the frequency of Score values for the test dataset 
test_scores = pd.Series(test_labels)
test_frequencies = test_scores.value_counts()
test_frequencies = test_frequencies.reset_index()
test_frequencies.columns = ["Score", "Frequency"]

plt.figure(figsize = (6, 4), dpi = 100)
plt.bar(test_frequencies["Score"], test_frequencies["Frequency"], color = "skyblue", edgecolor = "black")
plt.xlabel("Score")
plt.ylabel("Frequency")
plt.title("Frequency of Food Review Scores - Test Dataset")
ax = plt.gca()
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
plt.show()


## Implementation - Task 2
Implement a Naïve Bayes model for predicting the rating of a food review. Train your model on the training set and test it on the test set. Use an appropriate text representation. (**5%**)

In [ ]:
if (load_NB != True):
    
    # Naive Bayes model - input is converted to TF-IDF vectors and then Multinomial Naive Bayes is used 
    nb_model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    
    # Tune model for best value of alpha (lapace smoothing parameter) using 5-fold cross-validation 
    # More coarse grid searching was undertaken initially to narrow down to the below range 
    grid = {"multinomialnb__alpha" : [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]}
    
    grid_search = GridSearchCV(nb_model, grid, cv = 5, scoring = "accuracy")
    
    # TODO: Decide if F1-Score better 
    
    # TODO: Decide if adding fit_prior to the grid search makes sense 
    
    grid_search.fit(train_data, train_labels)
    
    best_alpha = grid_search.best_params_["multinomialnb__alpha"]
    
    print(best_alpha)
    
    nb_model.set_params(multinomialnb__alpha = best_alpha)
    
    # Train model 
    nb_model.fit(train_data, train_labels)
    
    # Save model 
    nb_model_filename = "nb_model.pkl"
    
    with open(nb_model_filename, "wb") as file:
        
        pickle.dump(nb_model, file)

if (load_NB == True):
    
    # Load model 
    with open("nb_model.pkl", "rb") as file:
        
        nb_model = pickle.load(file)
    
# Test model 
nb_predictions = nb_model.predict(test_data)


In [ ]:
# Plot the confusion matrix
score_catagory_labels = [1, 2, 3, 4, 5]

matrix = confusion_matrix(test_labels, nb_predictions)
sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
           xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.show()

# Compute and print classification performance metrics
print("Accuracy:\t%f" % accuracy_score(test_labels, nb_predictions))
print("F1-score:\t%f" % f1_score(test_labels, nb_predictions, average = "macro"))
print("Precision:\t%f" % precision_score(test_labels, nb_predictions, average = "macro"))
print("Recall:\t\t%f" % recall_score(test_labels, nb_predictions, average = "macro"))
print("\nClassification performance:\n%s" % classification_report(test_labels, nb_predictions))


## Implementation - Task 3
Implement a k-Nearest Neighbours model for predicting the rating of a food review. Train your model on the training set and test it on the test set. Use an appropriate text representation. You must select the best k by examining the performance of the model for $k \in \{1,3,5,7\}$, using an appropriate cross-validation approach. Create a plot for k vs. classification performance to justify your choice. (**10%**)

In [ ]:
if (load_precalculated_w2v_embeddings_300 != True):
    
    # Train word2vec word embedding model using train data to create denser vectors  
    w2v_word_embeddings_300 = gensim.models.Word2Vec(train_data_tokenised, vector_size = 300, window = 5, 
                                                 min_count = 1, sg = 0, seed = 123)
        
    # Save word2vec word embeddings 
    w2v_word_embeddings_300.save("w2v_word_embeddings_300.model")
    
# TODO: Explain why vector_size and window chosen 

if (load_precalculated_w2v_embeddings_300 == True):
    
    # Load word2vec word embeddings 
    w2v_word_embeddings_300 = Word2Vec.load("w2v_word_embeddings_300.model")
    

def text_to_single_vector(text, w2v_word_embeddings):
    """Function to trasform document (a review) in the form of a list of words into a single vector 
    representation which is the average vector representation of the embedded words in the review."""
    
    # Check text input is a list  
    if (not isinstance(text, list)):
        
        raise Exception("text input is not a list.")
    
    vector = sum(w2v_word_embeddings.wv[word] for word in text if word in w2v_word_embeddings.wv) / len(text)
    
    return vector


if (load_precalculated_w2v_docs_vectorised != True):
    
    # Vectorise train data into dense vector document representations 
    train_data_w2v_docs_vectorised = [text_to_single_vector(text, w2v_word_embeddings_300) for text in train_data_tokenised]
    
    # Vectorise test data into dense vector document representations 
    test_data_w2v_docs_vectorised = [text_to_single_vector(text, w2v_word_embeddings_300) for text in test_data_tokenised]
    
    # Save vectorised document representations 
    with open("train_data_w2v_docs_vectorised.pkl", "wb") as file:
        
        pickle.dump(train_data_w2v_docs_vectorised, file)
        
    # Save vectorised document representations 
    with open("test_data_w2v_docs_vectorised.pkl", "wb") as file:
        
        pickle.dump(test_data_w2v_docs_vectorised, file)
        
if (load_precalculated_w2v_docs_vectorised == True):
    
    # Load vectorised document representations 
    with open("train_data_w2v_docs_vectorised.pkl", "rb") as file:
    
        train_data_w2v_docs_vectorised = pickle.load(file)
    
    # Load vectorised document representations 
    with open("test_data_w2v_docs_vectorised.pkl", "rb") as file:
    
        test_data_w2v_docs_vectorised = pickle.load(file)
    
if (run_kNN == True):
    
    if (run_kNN_cross_validation == True):
        
        # Values of k to test 
        k_values = [1, 3, 5, 7]
        
        mean_accuracies = []
        
        for k in k_values:
            
            knn_model = KNeighborsClassifier(n_neighbors = k)
            
            scores = cross_val_score(knn_model, train_data_w2v_docs_vectorised, train_labels, cv = 5, scoring = "accuracy")
            
            # TODO: Explain why Accuracy vs F1-Score - If doing class balanced sampling then doesn't matter, 
            # if not then F1-Score makes most sense because of class imbalance 
            
            # TODO: Decide on if to tune for weights and metric parameters too 
            
            mean_accuracy = scores.mean()
            
            mean_accuracies.append(mean_accuracy)
            
        # Save cross-validation accuracies  
        with open("knn_mean_accuracies.pkl", "wb") as file:
            
            pickle.dump(mean_accuracies, file)
            
    if (run_kNN_cross_validation != True): # TODO: Run kNN training again to save mean accuracies 
        
        # Load cross-validation accuracies  
        with open("knn_mean_accuracies.pkl", "rb") as file:
            
            mean_accuracies = pickle.load(file)
    

In [ ]:
if (run_kNN):
    
    k_values = [1, 3, 5, 7]
    
    plt.figure(figsize = (6, 4), dpi = 100)
    plt.plot(k_values, mean_accuracies, marker = "o", linestyle = "-", color = "b")
    plt.title("kNN Model Performance by Number of Neighbours")
    plt.xlabel("Number of Neighbours (k)")
    plt.ylabel("Mean Accuracy")
    plt.xticks(k_values)
    plt.grid(True)
    ax = plt.gca()
    ax.spines["top"].set_color("black")
    ax.spines["right"].set_color("black")
    ax.spines["left"].set_color("black")
    ax.spines["bottom"].set_color("black")
    plt.savefig("Figure_2.png")
    plt.show()


In [ ]:
if (run_kNN == True):
    
    if (load_kNN != True):
        
        k_values = [1, 3, 5, 7]
        
        # k Nearest Neighbours model - input is converted to TF-IDF vectors and then kNN is used 
        # The k value which produced the hgihest accuracy from cross-validation on the train/validation set is used 
        knn_model = KNeighborsClassifier(n_neighbors = k_values[mean_accuracies.index(max(mean_accuracies))])
        
        # Train model 
        knn_model.fit(train_data_w2v_docs_vectorised, train_labels)
        
        # Save model 
        knn_model_filename = "knn_model.pkl"
        
        with open(knn_model_filename, "wb") as file:
            
            pickle.dump(knn_model, file)
            
    if (load_kNN == True):
        
        # Load model 
        with open("knn_model.pkl", "rb") as file:
            
            knn_model = pickle.load(file)
            
    # Test model 
    knn_predictions = knn_model.predict(test_data_w2v_docs_vectorised)


In [ ]:
if (run_kNN == True):
    
    # Plot the confusion matrix
    score_catagory_labels = [1, 2, 3, 4, 5]
    
    matrix = confusion_matrix(test_labels, knn_predictions)
    sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
                xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")
    plt.show()
    
    # Compute and print classification performance metrics
    print("Accuracy:\t%f" % accuracy_score(test_labels, knn_predictions))
    print("F1-score:\t%f" % f1_score(test_labels, knn_predictions, average = "macro"))
    print("Precision:\t%f" % precision_score(test_labels, knn_predictions, average = "macro"))
    print("Recall:\t\t%f" % recall_score(test_labels, knn_predictions, average = "macro"))
    print("\nClassification performance:\n%s" % classification_report(test_labels, knn_predictions))


## Implementation - Task 4
Implement a Convolutional Neural Network (CNN) model for predicting the rating of a food review. The model must have at least two convolutional layers. Train your model on the training set and test it on the test set. Use an appropriate text representation. (**13%**)

In [ ]:
# Calculate the number of words in each review after preprocessing etc. 
all_data_tokenised = train_data_tokenised + test_data_tokenised

n_words = [len(text) for text in all_data_tokenised]

# Summary statistics for the number of words per review 
pd.Series(n_words).describe()

plt.figure(figsize = (6, 4), dpi = 100)
plt.hist(n_words, bins = range(0, 2100, 50), color = "skyblue", edgecolor = "black")
plt.title("Distribution of Number of Words per Food Review after Preprocessing")
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.grid(axis = "y", alpha = 0.75)
plt.xlim([-30, 400])
ax = plt.gca()
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
plt.savefig("Figure_3.png")
plt.show()

max_words = round(pd.Series(n_words).describe()["max"])
print(f"Note: y axis limited to show only visible columns. Number of words values go as high as {max_words}.")


In [ ]:
# TODO: Try not removing stop words and see if that increases accuracy 

In [ ]:
def document_to_matrix(document, w2v_word_embeddings, max_length):
    """Function to trasform document (a review) in the form of a list of words into a matrix of 
    vector representations, until the specified max length."""
    
    # TODO: Add checks 
    
    matrix = np.zeros((max_length, 50))
    
    for i, word in enumerate(document):
        
        if i >= max_length:
            
            break
            
        if word in w2v_word_embeddings.wv:
            
            matrix[i] = w2v_word_embeddings.wv[word]
            
    return matrix
    
    
if (load_precalculated_w2v_embeddings_50 != True):
    
    # Train word2vec word embedding model using train data to create denser vectors  
    w2v_word_embeddings_50 = gensim.models.Word2Vec(train_data_tokenised, vector_size = 50, window = 5, 
                                                 min_count = 1, sg = 0, seed = 123)
        
    # Save word2vec word embeddings 
    w2v_word_embeddings_50.save("w2v_word_embeddings_50.model")
    
# TODO: Explain why vector_size and window chosen 

if (load_precalculated_w2v_embeddings_50 == True):
    
    # Load word2vec word embeddings 
    w2v_word_embeddings_50 = Word2Vec.load("w2v_word_embeddings_50.model")
    

In [ ]:
if (load_precalculated_w2v_words_vectorised != True):
    
    # Vectorise train data into dense vector word representations and combine into a 
    # list of dense matrix document representations
    train_data_w2v_words_vectorised = [document_to_matrix(text, w2v_word_embeddings_50, 50) for text in train_data_tokenised]
    
    # Vectorise test data into dense vector word representations and combine into a 
    # list of dense matrix document representations
    test_data_w2v_words_vectorised = [document_to_matrix(text, w2v_word_embeddings_50, 50) for text in test_data_tokenised]
    
    # TODO: Explain why 100 max length 
    
    # Save vectorised word representations 
    with open("train_data_w2v_words_vectorised.pkl", "wb") as file:
        
        pickle.dump(train_data_w2v_words_vectorised, file)
        
    # Save vectorised word representations 
    with open("test_data_w2v_words_vectorised.pkl", "wb") as file:
        
        pickle.dump(test_data_w2v_words_vectorised, file)

if (load_precalculated_w2v_words_vectorised == True):
    
    if (load_CNN != True | load_LSTM != True):
        
        # Load vectorised word representations 
        with open("train_data_w2v_words_vectorised.pkl", "rb") as file:
        
            train_data_w2v_words_vectorised = pickle.load(file)
    
    # Load vectorised word representations 
    with open("test_data_w2v_words_vectorised.pkl", "rb") as file:
    
        test_data_w2v_words_vectorised = pickle.load(file)
        

In [ ]:
if (load_CNN != True):
    
    embedding_size = (50, 50)
    optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0002)
    
    cnn_model = models.Sequential()
    
    cnn_model.add(layers.Conv1D(filters = 128, kernel_size = 5, activation = "relu", input_shape = embedding_size))
    cnn_model.add(layers.MaxPooling1D(pool_size = 5))
    cnn_model.add(layers.Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
    cnn_model.add(layers.GlobalMaxPooling1D())
    cnn_model.add(layers.Dense(units = 128, activation = "relu"))
    cnn_model.add(layers.Dense(5, activation = "softmax"))
    
    cnn_model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimiser, metrics = ["accuracy"])

train_labels_minus1 = [score - 1 for score in train_labels]

test_labels_minus1 = [score - 1 for score in test_labels]

if (load_CNN != True):
    
    epochs = 10
    
    early = EarlyStopping(monitor = "val_accuracy", patience = 3, restore_best_weights = True, mode = "auto")
    
    x_train = np.array(train_data_w2v_words_vectorised)
    y_train = np.array(train_labels_minus1)
    
    cnn_history = cnn_model.fit(x = x_train, y = y_train, validation_split = 0.3, epochs = epochs, callbacks = [early], verbose = 2)
    
    # Save model 
    cnn_model_filename = "cnn_model_3.pkl"
    
    with open(cnn_model_filename, "wb") as file:
        
        pickle.dump(cnn_model, file)
        
    # Save training history 
    cnn_model_history_filename = "cnn_model_history_3.pkl"
    
    with open(cnn_model_history_filename, "wb") as file:
        
        pickle.dump(cnn_history, file)
        
if (load_CNN == True):
    
    # Load model 
    with open("cnn_model_3.pkl", "rb") as file:
        
        cnn_model = pickle.load(file)
        
    # Load training history 
    with open("cnn_model_history_3.pkl", "rb") as file:
        
        cnn_history = pickle.load(file)
        

In [ ]:
plt.style.use("ggplot")
fig, (ax1, ax2) = plt.subplots(2, figsize = (8, 6), dpi = 100, sharex = True)
fig.suptitle("Convolutional Neural Network Training", fontsize = 14, color = "black")
ax1.set_ylim([0, 1.01])
ax1.plot(cnn_history.history["val_accuracy"], "b")
ax1.set_ylabel("Accuracy", color = "black")
ax1.plot(cnn_history.history["accuracy"], "r")
ax1.legend(["Validation", "Training"])
ax1.spines["top"].set_color("black")
ax1.spines["right"].set_color("black")
ax1.spines["left"].set_color("black")
ax1.spines["bottom"].set_color("black")
ax1.tick_params(axis = "x", colors = "black")
ax1.tick_params(axis = "y", colors = "black")
ax2.set_ylim([0, 1])
ax2.plot(cnn_history.history["val_loss"], "b")
ax2.plot(cnn_history.history["loss"], "r")
ax2.set_ylabel("Loss", color = "black")
ax2.set_xlabel("Epochs", color = "black")
ax2.legend(["Validation", "Training"])
ax2.spines["top"].set_color("black")
ax2.spines["right"].set_color("black")
ax2.spines["left"].set_color("black")
ax2.spines["bottom"].set_color("black")
ax2.tick_params(axis = "x", colors = "black")
ax2.tick_params(axis = "y", colors = "black")
plt.savefig("Figure_4.png")
plt.show()


In [ ]:
# Test model 
cnn_predictions = cnn_model.predict(np.array(test_data_w2v_words_vectorised))

cnn_predictions_labels = np.argmax(cnn_predictions, axis = 1)


In [ ]:
# Plot the confusion matrix 
score_catagory_labels = [1, 2, 3, 4, 5]

matrix = confusion_matrix(test_labels_minus1, cnn_predictions_labels)
sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
            xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.show()

# Compute and print classification performance metrics
print("Accuracy:\t%f" % accuracy_score(test_labels_minus1, cnn_predictions_labels))
print("F1-score:\t%f" % f1_score(test_labels_minus1, cnn_predictions_labels, average = "macro"))
print("Precision:\t%f" % precision_score(test_labels_minus1, cnn_predictions_labels, average = "macro"))
print("Recall:\t\t%f" % recall_score(test_labels_minus1, cnn_predictions_labels, average = "macro"))
print("\nClassification performance:\n%s" % classification_report(test_labels_minus1, cnn_predictions_labels))


## Implementation - Task 5
Implement a Recurrent Neural Network (RNN) or a Long Short-Term Memory (LSTM) model for predicting the rating of a food review. The model must have at least two RNN/LSTM layers. Train your model on the training set and test it on the test set. Use an appropriate text representation. (**12%**)

In [ ]:
if (load_LSTM != True):
    
    embedding_size = (50, 50)
    optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0002)
    
    lstm_model = models.Sequential()
    
    lstm_model.add(layers.Masking(mask_value = 0., input_shape = embedding_size))
    lstm_model.add(layers.Bidirectional(layers.LSTM(units = 64, return_sequences = True)))
    lstm_model.add(layers.Bidirectional(layers.LSTM(units = 64, go_backwards = True, dropout = 0.2)))
    lstm_model.add(layers.Dense(units = 64, activation = "relu"))
    lstm_model.add(layers.Dense(5, activation = "softmax"))
    
    lstm_model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimiser, metrics = ["accuracy"])

train_labels_minus1 = [score - 1 for score in train_labels]

test_labels_minus1 = [score - 1 for score in test_labels]

if (load_LSTM != True):
    
    epochs = 10
    
    early = EarlyStopping(monitor = "val_accuracy", patience = 3, restore_best_weights = True, mode = "auto")
    
    x_train = np.array(train_data_w2v_words_vectorised)
    y_train = np.array(train_labels_minus1)
    
    lstm_history = lstm_model.fit(x = x_train, y = y_train, validation_split = 0.3, epochs = epochs, callbacks = [early], verbose = 2)
    
    # Save model 
    lstm_model_filename = "lstm_model_2.pkl"
    
    with open(lstm_model_filename, "wb") as file:
        
        pickle.dump(lstm_model, file)
        
    # Save training history 
    lstm_model_history_filename = "lstm_model_history_2.pkl"
    
    with open(lstm_model_history_filename, "wb") as file:
        
        pickle.dump(lstm_history, file)
        
if (load_LSTM == True):
    
    # Load model 
    with open("lstm_model_2.pkl", "rb") as file:
        
        lstm_model = pickle.load(file)
        
    # Load training history 
    with open("lstm_model_history_2.pkl", "rb") as file:
        
        lstm_history = pickle.load(file)
        

In [ ]:
plt.style.use("ggplot")
fig, (ax1, ax2) = plt.subplots(2, figsize = (8, 6), dpi = 100, sharex = True)
fig.suptitle("Long Short-Term Memory Model Training", fontsize = 14, color = "black")
ax1.set_ylim([0, 1.01])
ax1.plot(lstm_history.history["val_accuracy"], "b")
ax1.set_ylabel("Accuracy", color = "black")
ax1.plot(lstm_history.history["accuracy"], "r")
ax1.legend(["Validation", "Training"])
ax1.spines["top"].set_color("black")
ax1.spines["right"].set_color("black")
ax1.spines["left"].set_color("black")
ax1.spines["bottom"].set_color("black")
ax1.tick_params(axis = "x", colors = "black")
ax1.tick_params(axis = "y", colors = "black")
ax2.set_ylim([0, 1])
ax2.plot(lstm_history.history["val_loss"], "b")
ax2.plot(lstm_history.history["loss"], "r")
ax2.set_ylabel("Loss", color = "black")
ax2.set_xlabel("Epochs", color = "black")
ax2.legend(["Validation", "Training"])
ax2.spines["top"].set_color("black")
ax2.spines["right"].set_color("black")
ax2.spines["left"].set_color("black")
ax2.spines["bottom"].set_color("black")
ax2.tick_params(axis = "x", colors = "black")
ax2.tick_params(axis = "y", colors = "black")
plt.savefig("Figure_5.png")
plt.show()


In [ ]:
# Test model 
lstm_predictions = lstm_model.predict(np.array(test_data_w2v_words_vectorised))

lstm_predictions_labels = np.argmax(lstm_predictions, axis = 1)


In [ ]:
# Plot the confusion matrix 
score_catagory_labels = [1, 2, 3, 4, 5]

matrix = confusion_matrix(test_labels_minus1, lstm_predictions_labels)
sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
            xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.show()

# Compute and print classification performance metrics
print("Accuracy:\t%f" % accuracy_score(test_labels_minus1, lstm_predictions_labels))
print("F1-score:\t%f" % f1_score(test_labels_minus1, lstm_predictions_labels, average = "macro"))
print("Precision:\t%f" % precision_score(test_labels_minus1, lstm_predictions_labels, average = "macro"))
print("Recall:\t\t%f" % recall_score(test_labels_minus1, lstm_predictions_labels, average = "macro"))
print("\nClassification performance:\n%s" % classification_report(test_labels_minus1, lstm_predictions_labels))


## Implementation - Task 6
Compute the confusion matrix, accuracy, F1-score, precision and recall for each model. (10%)

In [ ]:
# Naive Bayes 

# Plot the confusion matrix
score_catagory_labels = [1, 2, 3, 4, 5]

matrix = confusion_matrix(test_labels, nb_predictions)
sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
           xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.savefig("Figure_6_2.png")
plt.show()

# Compute and print classification performance metrics
print("Accuracy:\t%f" % accuracy_score(test_labels, nb_predictions))
print("F1-score:\t%f" % f1_score(test_labels, nb_predictions, average = "macro"))
print("Precision:\t%f" % precision_score(test_labels, nb_predictions, average = "macro"))
print("Recall:\t\t%f" % recall_score(test_labels, nb_predictions, average = "macro"))
print("\nClassification performance:\n%s" % classification_report(test_labels, nb_predictions))


In [ ]:
# K Nearest Neighbours 

if (run_kNN == True):
    
    # Plot the confusion matrix
    score_catagory_labels = [1, 2, 3, 4, 5]
    
    matrix = confusion_matrix(test_labels, knn_predictions)
    sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
                xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")
    plt.savefig("Figure_7_2.png")
    plt.show()
    
    # Compute and print classification performance metrics
    print("Accuracy:\t%f" % accuracy_score(test_labels, knn_predictions))
    print("F1-score:\t%f" % f1_score(test_labels, knn_predictions, average = "macro"))
    print("Precision:\t%f" % precision_score(test_labels, knn_predictions, average = "macro"))
    print("Recall:\t\t%f" % recall_score(test_labels, knn_predictions, average = "macro"))
    print("\nClassification performance:\n%s" % classification_report(test_labels, knn_predictions))


In [ ]:
# Convolutional Neural Network 

# Plot the confusion matrix 
score_catagory_labels = [1, 2, 3, 4, 5]

matrix = confusion_matrix(test_labels_minus1, cnn_predictions_labels)
sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
            xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.savefig("Figure_8_2.png")
plt.show()

# Compute and print classification performance metrics
print("Accuracy:\t%f" % accuracy_score(test_labels_minus1, cnn_predictions_labels))
print("F1-score:\t%f" % f1_score(test_labels_minus1, cnn_predictions_labels, average = "macro"))
print("Precision:\t%f" % precision_score(test_labels_minus1, cnn_predictions_labels, average = "macro"))
print("Recall:\t\t%f" % recall_score(test_labels_minus1, cnn_predictions_labels, average = "macro"))
print("\nClassification performance:\n%s" % classification_report(test_labels_minus1, cnn_predictions_labels))


In [ ]:
# Long Short-Term Memory model 

# Plot the confusion matrix 
score_catagory_labels = [1, 2, 3, 4, 5]

matrix = confusion_matrix(test_labels_minus1, lstm_predictions_labels)
sns.heatmap(matrix.T, square = True, annot = True, fmt = "d", 
            xticklabels = score_catagory_labels, yticklabels = score_catagory_labels)
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.savefig("Figure_9_2.png")
plt.show()

# Compute and print classification performance metrics
print("Accuracy:\t%f" % accuracy_score(test_labels_minus1, lstm_predictions_labels))
print("F1-score:\t%f" % f1_score(test_labels_minus1, lstm_predictions_labels, average = "macro"))
print("Precision:\t%f" % precision_score(test_labels_minus1, lstm_predictions_labels, average = "macro"))
print("Recall:\t\t%f" % recall_score(test_labels_minus1, lstm_predictions_labels, average = "macro"))
print("\nClassification performance:\n%s" % classification_report(test_labels_minus1, lstm_predictions_labels))


## Implementation - Task 7
Store the **four** trained models in files and implement a function `predict_food_review(text, model)` that given a <ins>text string</ins> (“`text`”) and model <ins>filename</ins> (“`model`”), it will load the pre-trained model, and predict the food review rating of the input text. The function should be able to work without requiring to rerun all or part of your code. (**10%**)

In [ ]:
# Function works under the assumption that all the required packages above are loaded 

# TODO: Decide if loading packages within the function so it is completely self contained makes sense 

def clean_text(text):
    """Function to clean text input to ensure input strings are processed in the same way as for 
    model train and test data."""
    
    # Note: code here is different to preprocessing in the main script as the latter is for processing whole 
    # pandas series and the former is individual stings. The results are identical. 
    
    # Check text input is a string 
    if (not isinstance(text, str)):
        
        raise Exception("text input is not a string.")
    
    # Transform text to all lowercase 
    text = text.lower()
    
    # Remove all "-", ".", "'" and replace with "" e.g. lower-case to lowercase, U.K to UK, don't to dont 
    text = re.sub(r"[-.']", "", text)
    
    # Replace html tags with " " 
    text = re.sub(r"<.+?>", " ", text)
    
    # Replace happy emojis with " happy " 
    text = re.sub(r"[:][)]|[(][:]", " happy ", text)
    
    # Replace sad emojis with " sad " 
    text = re.sub(r"[:][(]|[)][:]", " sad ", text)
    
    # Remove punctuation 
    text = re.sub(r"[^\w\s]", " ", text)
    
    return text


def predict_food_review(text, model):
    """Function to predict the Score (1 to 5) of a food review using pretrained Naive Bayes, k Nearest Neighbours, 
    Convolutional Neural Network, and Long Short-Term Memory models. Provide input text as a string. 
    Choose models from: "nb_model.pkl", "knn_model.pkl", "cnn_model.pkl", and "lstm_model.pkl"."""
    
    # TODO: Make variable and file naming consistent 
    
    # Check text input is a string 
    if (not isinstance(text, str)):
        
        raise Exception("text input is not a string.")
        
    # Check model filename input is a string 
    if (not isinstance(model, str)):
        
        raise Exception("model filename input is not a string.")
        
    # Check a valid model filename input is provided 
    valid_model_filenames = ["nb_model.pkl", "knn_model.pkl", "cnn_model.pkl", "lstm_model.pkl"]
    
    if (not model in valid_model_filenames):
        
        raise Exception("model filename input is not a valid option. \nValid options include: \n'nb_model.pkl', 'kNN_model.pkl', 'cnn_model.pkl', and 'lstm_model.pkl'")
    
    # Clean text in the same way as for initial train and test 
    text = clean_text(text)
    
    # POS tag, remove stop words, and lemmatise in the same way as for initial train and test and return a list of lemmas 
    text = remove_stopwords_and_lemmatise(text)
    
    # Naive Bayes 
    if (model == "nb_model.pkl"):
        
        # Join lists of lemmas to string 
        text = " ".join(text)
        
        # Make text an element of a list - required for predict() 
        text = [text]
        
        # Load Naive Bayes model 
        with open("nb_model.pkl", "rb") as file:
            
            nb_model = pickle.load(file)
            
        # Returns np array 
        predicted_score = nb_model.predict(text)
        
        # Extract element from array and convert to string 
        predicted_score = str(predicted_score.item())
        
        return predicted_score
    
    # k Nearest Neighbours 
    if (model == "knn_model.pkl"):
        
        # Load word2vec word embeddings 
        w2v_word_embeddings_300 = Word2Vec.load("w2v_word_embeddings_300.model")
        
        # Vectorise text into a dense vector document representation 
        text = [text_to_single_vector(text, w2v_word_embeddings_300)]
        
        # Load k Nearest Neighbours 
        with open("knn_model.pkl", "rb") as file:
            
            knn_model = pickle.load(file)
            
        # Returns np array 
        predicted_score = knn_model.predict(text)
        
        # Extract element from array and convert to string 
        predicted_score = str(predicted_score.item())
        
        return predicted_score
        
    # Convolutional Neural Network 
    if (model == "cnn_model.pkl"):
        
        # Load word2vec word embeddings 
        w2v_word_embeddings_50 = Word2Vec.load("w2v_word_embeddings_50.model")
        
        # Vectorise text data into dense vector word representations 
        text = np.array([document_to_matrix(text, w2v_word_embeddings_50, 50)])
        
        # Load Convolutional Neural Network 
        with open("cnn_model.pkl", "rb") as file:
            
            cnn_model = pickle.load(file)
            
        # Returns np array 
        predicted_score = cnn_model.predict(text)
        predicted_score = np.argmax(predicted_score, axis = 1)
        
        # Extract element from array and convert to string 
        predicted_score = str(predicted_score.item() + 1) # Add 1 to turn 0 indexed predictions into 1-5 predictions 
        
        return predicted_score
    
    # Long Short-Term Memory model 
    if (model == "lstm_model.pkl"):
        
        # Load word2vec word embeddings 
        w2v_word_embeddings_50 = Word2Vec.load("w2v_word_embeddings_50.model")
        
        # Vectorise text data into dense vector word representations 
        text = np.array([document_to_matrix(text, w2v_word_embeddings_50, 50)])
        
        # Load Convolutional Neural Network 
        with open("lstm_model.pkl", "rb") as file:
            
            lstm_model = pickle.load(file)
            
        # Returns np array 
        predicted_score = lstm_model.predict(text)
        predicted_score = np.argmax(predicted_score, axis = 1)
        
        # Extract element from array and convert to string 
        predicted_score = str(predicted_score.item() + 1) # Add 1 to turn 0 indexed predictions into 1-5 predictions 
        
        return predicted_score
    

In [ ]:
#stopwords.words("english") # TODO: Remove negating stop words? 

# TODO: Ensure final function loads correct model versions and remove all _1 and _2 versions from working directory 

## Report - Task 1
Critical discussion about the dataset (suitability, problems, class balance, etc.). (**6%**)

### Dataset

The food review dataset used for this report contained 3 features (columns) and 540,031 observations (rows), with 162,009 duplicate rows. The dataset was deemed large enough to undertake meaningful sentiment analysis with a large vocabulary of words in the dataset in theory allowing models trained on it to generalise well to unseen data. The dataset was large enough that both the space and time complexity of data processing and modelling needed to be carefully considered to prevent running out of memory and prohibitively long run times respectively. The datasets 3 features were "Score": the food review score, "Summary": a summary of the food review, and "Text" the food review itself. There were no missing values in the Score or Text columns. There were 27 missing values in the Summary column, however this was not assessed to be an issue as the primary columns required were Score and Text. There were no unexpected values in the Score column (only 1-5 scores), however there was class imbalance present with scores of 5 far more prevelant than other scores which could potentially bias models to the majority class, see Figure 1. The Text column contained raw text which had not been preprocessed or tokenised, containing: HTML tags, punctuation, emojis, capital letters, etc. 

While the potential problems detailed above could provide challenges for undertaking sentiment analysis, it was determined that with effective mitigation the dataset was suitable for undertaking the task of predicting food review scores. 

__Figure 1.__
![Figure 1.](Figure_1.png)


- Dataset has 3 features and 540,031 observations - this is large enough for meaningful sentiment analysis to be undertaken and large enough that the computational complexity of data processing and modelling methods needed to be considered to prevent prohibitively long run times. 
- No missing values in Score column. 
- No missing values in Text column. 
- 27 missing values in Summary column - this doesn't really matter though because we primarily care about Text and Score. 
- No unexpected values in the Score column - only 1, 2, 3, 4, 5. 
- Dataset has 162,009 duplicate rows. 
- The Text column contains raw text which is not tokenised or preprocessed: HTML tags and punctuation which need removed, emojis such as smiley faces which need removed or transformed, and raw text containing capital letters which needs to be lowercased. 
- Class imbalance with majority of observations with a Score of 5. 

## Report - Task 2
Description and justification of the data preparation step(s) used. (**6%**)

### Data Preparation

The dataset was deduplicated to prevent the erroneous repeated rows biasing models trained on the dataset. HTML tags were removed to reduce noise. Happy and sad emojis were tranformed into "happy" and "sad" to retain useful meaning post punctuation removal. "-", ".", and "'" were removed and replaced with "" to ensure consistency on the representation of words e.g. lower-case vs lowercase, U.K. vs UK, don't vs dont. All punctuation was then removed and replaced with " " and all text lowercased. All these processing steps were undertaken on the Summary and Text columns to reduce noise and non-useful information. This also reduces the computational complexity due to the denser vector representations achievable by not having repeated versions of words e.g. capitalised vs not. The Summary and Text columns were then concatenated, as from manual analysis is was determined that the Summary column also contained useful information regarding the sentiment of reviews. 

The text was then tokenised. Stop words were removed to reduce noise and remove potentially uninformative words. This also allowed for denser vector representations to make data processing and modelling less computationally intensive, while aiming to retain useful information. A trade-off was made here with the cost of reduced computational complexity being a potential reduction in models accuracy, e.g. the removal of certain stop words like "not" could mean models aren't able to capture important negation information. Penn Treeback Part of Speech (POS) tagging was then applied, with tags subsequently converted to simpler WordNet POS tags. This simplification could result in a reduction in models accuracy with only Nouns, Verbs, Adverbs, and Adjectives as classes for tagging, with other word types defaulted to Nouns. Words were then lemmatised to allow for denser vector representations to make data processing and modelling less computationally intensive while aiming to retain words general meanings, e.g. "eats", "eating" to "eat". 

The dataset was split into train and test sets for modelling using a 70/30 train/test split and stratified random sampling by Score to ensure both sets had a similar distribution of Score values. Given the size of the dataset simple random sampling did generally provide similar distributions of Score values in the train and test set but it was decided it was better to guarentee a representative split. 

The lemmatised text was then used to calculate each of the different word or document embeddings used as inputs for the various models in this report, detailed below. 


- Deduplication to prevent the data biasing the models due to erroneous repeated rows. 
- HTML tag removal, emoji transformation into words, lowercasing, -, ., and ' removal with no space, and punctuation removal to remove noise and non-useful information from the Text. This helps computationally because it will lead to denser vector representations of the text due to not repeating capitalised vs not versions of words etc. This was undertaken on both Summary and Text columns. 
- The Summary and Text columns were concatenated as from manual checking it was determined that the Summary column also contained useful information regarding the sentiment of reviews. 
- The text was then tokenised. 
- Stop words were removed to remove noise / potentially uniformative words and to create denser vectors making the processing and modelling less computationally intesive, given the relatively large dataset. A standard set of English stopwords was used from the NLTK package. There is potentially some drop in modelling accuracy in exchange for the reduced computational complexity as words such as "not", "wasn't" etc. were removed which could negate positive words and change the sentiment of reviews. 
- Penn Treeback POS tagging was then applied with tags subsequently converted to simpler WordNet POS tags. This simplification could result in a reduction in accuracy with only Nouns, Verbs, Adverbs, and Adjectives as classes for tagging and if no mapping was available words were defaulted to Nouns. 
- Words were then lemmatised to create denser vectors making the processing and modelling less computationally intesive, given the relatively large dataset. 
- Further processing of the tokenised / lemmatised text was then undertaken for the different representations used by the different models, with this processing detailed below. 

## Report - Task 3
Description and commentary on the machine learning architectures used, including a description and justification of the text representation method(s) used. (**7%**)

### Machine Learning Models

#### Naive Bayes 



#### k Nearest Neighbours 



#### Convolutional Neural Network 



#### Long Short-Term Memory Model 




#### Naive Bayes 

- Term Frequency-Inverse Document Freuency (TF-IDF) vector word embedding was used due to its compatibility with Naive Bayes, with each document represented as a vector and each vector element representing a specific words TF-IDF score. TF-IDF weights words based on how often they appear across all documents allowing the Naive Bayes model to focus on terms which should be more impactful in predicting the sentiment of a review and reducing the impact of common words which should be less impactful. 
- Multinomial Naive Bayes was chosen given the 5 classes for prediction. 
- Laplace Smoothing was used to address the problem of zero probabilities and 5-fold cross-validation was used to choose the best alpha value, using accuracy as the performance metric compared. This helped prevent the model from predicting almost exclusively scores of 5 due to the class imbalance in the dataset. 

#### k Nearest Neighbours 

- Word2Vec Continuous Bag of Words word embedding was used to create 300 dimensional dense vector representations of each word and then these were used to create dense vector representations of each review (document). Each word was transformed into a 300 length vector representation, 300 was chosen as a trade-off between capturing as much information as possible about each word and the added computational complexity of higher dimensional vectors. The context window was chosen as 5 to capture the context of the 5 words either side of any specific word to learn from. The dense vector representations for the words in a review were then averaged to get a dense document representation (300 dimensional vector) to capture the average sentiment in each review. A single document representation was chosen given that it reduces the computational complexity when using k Nearest Neighbours relative to sparse vector representation or dense vector representation per word formats. 
- 5-fold cross-validation was used to choose the best value of k (from 1, 3, 5, 7), using accuracy as the performance metric compared. A k value of 7 was determined to have the highest average accuracy in cross-validation, see Figure 2. 

__Figure 2.__
![Figure 2.](Figure_2.png)

#### Convolutional Neural Network 

- Word2Vec Continuous Bag of Words word embedding was used to created 50 dimensional dense vector representations of each word and then these were used to create a matrix of dense vector word representations for each review (50 by 50). 50 diminsions was chosen to reduce the space complexity of the Word2Vec embeddings and to reduce the time complexity for model training. The distribution of words per review was analysed and it was determined that 50 words would capture all the words in just under 75% or all reviews, see Figure 3. Matrices for reviews with less than 50 words were padded with zeros and reviews with more than 50 words were truncated to 50 to ensure that the matrix representations for all reviews had the same dimensions. This was determined to be a good trade-off between the amount of information able to be captured in each review matrix and the increased space and time complexity of larger matrices. 

__Figure 3.__
![Figure 3.](Figure_3.png)

- The model arhcitecture used a 50 by 50 input shape feeding into a 1-dimensional convolutional layer with a filter size of 128, kernal size of 5 and ReLU activation function. The layer is designed to capture local patterns in the input with a 5 word context window and the 128 node filter size was chosen as a trade-off between being able to capture complex patterns in the data and computational complexity. A 1-dimensional max pooling layer follows to try and abstract the features extracted by the previous layer and prevent overfitting. A 1-dimensional convolutional layer identical to the first follows to try and further refine the features extracted from the input. A 1-dimensional global max pooling layer follows to reduce the dimensionality before connecting to the following dense layer which interprets the feature extracted and pooled by previous layers and contains 128 nodes and uses the ReLu activation function. All ReLU activation functions used are to introduce non-linearity to the model to allow it to capture non-linear relationships and ReLU is more computationally efficient than other non-linear options. The output layer is a dense layer with 5 nodes corresponding to the number of classes with a softmax activation function used to output the probability distribution over the 5 classes. 
- The loss function used was sparse categorical cross entropy due to the multi-class nature of the classification problem and the mutually expclusive classes. 
- The Adam optimiser was used as it is conventional and computationally efficient. 
- A 70/30 train/validation split was used for model training with a stoping condition on no accuracy improvement for 3 epochs to prevent unneccassary computation. 
- Accuracy was chosen as the performance metric for model training, however given the class imbalance present repeating the analysis using F1-Score might be worthwhile. 

#### Long Short-Term Memory Model 

- The same Word2Vec Continuous Bag of Words 50 by 50 matrices of dense vector word representations for reviews was used as described in the Convolutional Neural Network section for the same reasons described. 
- 


## Report - Task 4
Detailed performance evaluation of the trained machine learning models in terms of the computed performance metrics. (**5%**)

#### Naive Bayes 

__Figure 6.__
![Figure 6.](Figure_6.png)

__Table 1.__

| Performance Metric | Score |
|----------|----------|
| Accuracy: | 0.68 |
| F1-Score: | 0.31 |
| Precision: | 0.49 |
| Recall: | 0.30 |

Note: Macro-averaging used for F1-Score, Precision, and recall. 

Naive Bayes had an overall accuracy of 0.68, however this is due to the model primarily predicting Scores of 5 and the test dataset having majority 5 Scores. The model predicts 5 the majority of the time despite the true class and this is reflected in the low recall score 

#### k Nearest Neighbours 

__Figure 7.__
![Figure 7.](Figure_7.png)

__Table 2.__

| Performance Metric | Score |
|----------|----------|
| Accuracy: | 0.66 |
| F1-Score: | 0.37 |
| Precision: | 0.40 |
| Recall: | 0.36 |

Note: Macro-averaging used for F1-Score, Precision, and recall. 

#### Convolutional Neural Network 

__Figure 8.__
![Figure 8.](Figure_8.png)

__Table 3.__

| Performance Metric | Score |
|----------|----------|
| Accuracy: | 0.71 |
| F1-Score: | 0.41 |
| Precision: | 0.49 |
| Recall: | 0.41 |

Note: Macro-averaging used for F1-Score, Precision, and recall. 

#### Long Short-Term Memory Model 

__Figure 9.__
![Figure 9.](Figure_9.png)

__Table 1.__

| Performance Metric | Score |
|----------|----------|
| Accuracy: | 0.74 |
| F1-Score: | 0.50 |
| Precision: | 0.55 |
| Recall: | 0.49 |

Note: Macro-averaging used for F1-Score, Precision, and recall. 


## Report - Task 5
Critical discussion on the achieved results, including potential limitations and usage instructions/suggestions. (**6%**)

Potential improvements:

- Using a class balanced trianing set and testing on the test set 
- Using F1-Score for model selection?
- Increasing the number of nodes per layer in NNs 
- Including negation stop words to see if it increases performance 